## Web Scraping GitHub.io For Friends Scripts

### Import Libraries and get the URL

In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://fangj.github.io/friends/'
res = requests.get(url)

In [3]:
res.status_code #200-good to go

200

In [4]:
res.content[:50]

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Trans'

In [5]:
soup = BeautifulSoup(res.content)

In [6]:
soup.find('h3').text

'SEASON 1'

### Getting the Data

Here, I am scrapping the page for each season, episode and its title, with the hyperlink to the scripts for each episode.

In [32]:
seasons = []
for j in soup.find_all('ul'):
    for i in soup.find_all('li'):
        season = {}
        epi = (i.find('a').text).split(' ',1)
        season['season'] = epi[0][0]
        season['episode'] = epi[0][:]
        season['title'] = epi[-1].strip()
        season['href'] = i.find('a').attrs['href']
        seasons.append(season)

In [33]:
# seasons = []
# for i in soup.find_all('h3'):
#     print(soup.find_all('h3'))
#     for j in soup.find('ul').find_all('li'):
#         season = {}
#         season['season'] = i.text[-1]
#         print(i.text[-1])
#         epi = (j.find('a').text).split(' ',1)
#         print(epi)
#         season['episode'] = epi[0][1:]
#         print(epi[0][1:])
#         season['title'] = epi[-1]
#         season['href'] = j.find('a').attrs['href']
#         seasons.append(season)
#         print('done')

In [37]:
seasons[88:100]

[{'season': '4',
  'episode': '417',
  'title': 'The Free Porn',
  'href': 'season/0417.html'},
 {'season': '4',
  'episode': '418',
  'title': "Rachel's New Dress",
  'href': 'season/0418.html'},
 {'season': '4',
  'episode': '419',
  'title': 'All The Haste',
  'href': 'season/0419.html'},
 {'season': '4',
  'episode': '420',
  'title': 'The Wedding Dresses',
  'href': 'season/0420.html'},
 {'season': '4',
  'episode': '421',
  'title': 'The Invitation',
  'href': 'season/0421.html'},
 {'season': '4',
  'episode': '422',
  'title': 'The Worst Best Man Ever',
  'href': 'season/0422.html'},
 {'season': '4',
  'episode': '423',
  'title': "Ross's Wedding",
  'href': 'season/0423.html'},
 {'season': '5',
  'episode': '501',
  'title': 'Ross Says Rachel',
  'href': 'season/0501.html'},
 {'season': '5',
  'episode': '502',
  'title': 'All The Kissing',
  'href': 'season/0502.html'},
 {'season': '5',
  'episode': '503',
  'title': 'Hundredth',
  'href': 'season/0503.html'},
 {'season': '5',

Next, I need to go into each page and get the dialouge from each character and the name of the character who said it to complete my dataset!

In [40]:
lines = []

for i in range(len(seasons)):
    base_url = url + seasons[i].get('href')
    request = requests.get(base_url)
    soup1 = BeautifulSoup(request.content)
    
    #looking at align to left attr
    for rest in soup1.find_all('p', {'align': 'left'}): 
        #line = {}
        #line['season'] = seasons[i].get('season')
        #line['episode'] = seasons[i].get('episode')
        #line['title'] = seasons[i].get('title')
        if rest.find('b') != None:
            print(rest)
            print(rest.find('b').text.split(':')[0])
            #line['character'] = rest.find('b').text.split(':')[0]

            if rest.find('font') != None:
                print(seasons[i].get('episode'))
                print(rest.find('font').text.split(':'))
                print(rest.find('font').text.split(':')[1])
                #line['dialogue'] = rest.find('font').text.split(':')[1]
        #lines.append(line)
        

<p align="left"><font size="3"><b>Monica:</b> There's nothing to tell! He's just some guy
I work with!</font></p>
Monica
101
['Monica', " There's nothing to tell! He's just some guy\nI work with!"]
 There's nothing to tell! He's just some guy
I work with!
<p align="left"><font size="3"><b>Joey:</b> C'mon, you're going out with the guy! There's
gotta be something wrong with him!</font></p>
Joey
101
['Joey', " C'mon, you're going out with the guy! There's\ngotta be something wrong with him!"]
 C'mon, you're going out with the guy! There's
gotta be something wrong with him!
<p align="left"><font size="3"><b>Chandler:</b> <font color="#0000FF">All right Joey, be
nice.  </font>So does he have a hump? A hump and a hairpiece?</font></p>
Chandler
101
['Chandler', ' All right Joey, be\nnice.\xa0 So does he have a hump? A hump and a hairpiece?']
 All right Joey, be
nice.  So does he have a hump? A hump and a hairpiece?
<p align="left"><font size="3"><b>Phoebe:</b> Wait, does he eat chalk?</font>

IndexError: list index out of range

### Turning the Data into a Dataframe

In [31]:
friends_df = pd.DataFrame(lines)

In [33]:
friends_df.head(5)

,season,episode,title,character,dialogue
0,1,01,Monica Gets A Roommate,NaN,NaN
1,1,01,Monica Gets A Roommate,Monica,There's nothing to tell! He's just some guy\n...
2,1,01,Monica Gets A Roommate,Joey,"C'mon, you're going out with the guy! There's..."
3,1,01,Monica Gets A Roommate,Chandler,"All right Joey, be\nnice. So does he have a ..."
4,1,01,Monica Gets A Roommate,Phoebe,"Wait, does he eat chalk?"


In [47]:
friends_df['season'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'], dtype=object)

In [34]:
friends_df.shape

(5570, 5)

### Exporting the Friends Dataset

In [36]:
friends_df.to_csv('../Datasets/friends.csv', index=False)

In [37]:
friends_df.isnull().sum()

season         0
episode        0
title          0
character    920
dialogue     930
dtype: int64

In [39]:
friends_df.loc[(friends_df['dialogue'].isnull()) & (friends_df['character'].isnull())]

,season,episode,title,character,dialogue
0,1,01,Monica Gets A Roommate,NaN,NaN
5,1,01,Monica Gets A Roommate,NaN,NaN
9,1,01,Monica Gets A Roommate,NaN,NaN
11,1,01,Monica Gets A Roommate,NaN,NaN
20,1,01,Monica Gets A Roommate,NaN,NaN
...,...,...,...,...,...
5564,0,19,The Monkey Gets Away,NaN,NaN
5565,0,20,The Evil Orthodontist,NaN,NaN
5567,0,21,The Fake Monica,NaN,NaN
5568,0,22,The Ick Factor,NaN,NaN


In [40]:
friends_df.loc[(friends_df['dialogue'].isnull()) & (friends_df['character'].isnull() == False)]

,season,episode,title,character,dialogue
553,1,20,The Evil Orthodontist,Chandler,NaN
1110,2,20,The Evil Orthodontist,Chandler,NaN
1667,3,20,The Evil Orthodontist,Chandler,NaN
2224,4,20,The Evil Orthodontist,Chandler,NaN
2781,5,20,The Evil Orthodontist,Chandler,NaN
3338,6,20,The Evil Orthodontist,Chandler,NaN
3895,7,20,The Evil Orthodontist,Chandler,NaN
4452,8,20,The Evil Orthodontist,Chandler,NaN
5009,9,20,The Evil Orthodontist,Chandler,NaN
5566,0,20,The Evil Orthodontist,Chandler,NaN


In [44]:
friends_df.loc[(friends_df['title'] == ' The Evil Orthodontist ')]

,season,episode,title,character,dialogue
552,1,20,The Evil Orthodontist,NaN,NaN
553,1,20,The Evil Orthodontist,Chandler,NaN
1109,2,20,The Evil Orthodontist,NaN,NaN
1110,2,20,The Evil Orthodontist,Chandler,NaN
1666,3,20,The Evil Orthodontist,NaN,NaN
1667,3,20,The Evil Orthodontist,Chandler,NaN
2223,4,20,The Evil Orthodontist,NaN,NaN
2224,4,20,The Evil Orthodontist,Chandler,NaN
2780,5,20,The Evil Orthodontist,NaN,NaN
2781,5,20,The Evil Orthodontist,Chandler,NaN
